# Development processes for creating storm meteorological field for FVCOM
**Author: Jun Sasaki  Coded on November 6, 2021  Updated on January 1, 2022**<br>
- Creating a meteorological forcing file for FVCOM 4.4.2 with NetCDF-4.
- NetCDF-4 on ITO-A is available.
- Referred to [Honda & Sameshima (2018](https://www.ysk.nilim.go.jp/kenkyuseika/pdf/ks1039.pdf), [Tajima et al. (2016)](https://doi.org/10.1142/S0578563416400027), and [Liu and Sasaki (2019)](https://doi.org/10.1038/s41598-019-48728-7)
- For storm data, [besttracks](https://github.com/jsasaki-utokyo/besttracks) is used.

In [ ]:
import numpy as np
# from PyFVCOM.read import ncread  ## too slow
import PyFVCOM as pf
import netCDF4
from cftime import num2date, date2num
import cftime
import datetime
import pandas as pd
from dateutil.parser import parse
import sys
import os
from pyproj import Proj
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams['font.size'] =16
## Using besttracks
from besttracks import parseJMA
import warnings
warnings.simplefilter('ignore')
import pickle
%matplotlib inline

# Read FVCOM grid and set up coordinate variables
### Set FVCOM grid file (`True`) or output netCDF (`False`) and Casename_wnd.nc
- When **fvcom_grid** is read for the first time, it is serialized to **fvcom_grid_pickle**, which is to be deserialized quickly next time.
- `x[node], y[node], lon[node], lat[node], xc[nele], yc[nele], lonc[nele], latc[nele] nv[three, nele]`

In [ ]:
## Select FVCOM grid file (True) or output netCDF (False)
grid_is_fvcom = True
fvcom_nc = "tokyobay_futtsu1.0_0001.nc" ## optional
casename = "tb_futtsu"
fvcom_grid = casename + "_grd.dat"
fvcom_grid_pickle = casename + "_grd.pickle"
fvcom_wnd  = casename + "_wnd.nc"

### from FVCOM output netCDF
- `netCDF.Datase(fvcom_nc, 'r')`
- This is fast but needs FVCOM output netCDF

In [ ]:
if not grid_is_fvcom:
    ## Reading grids from FVCOM output netCDF, which is fast.
    print("Reading the FVCOM grid from FVCOM output netCDF")
    FVCOM = netCDF4.Dataset(fvcom_nc, 'r')
    ## FVCOM = ncread(fvcom_nc)  ## too slow
    x=FVCOM.variables['x'][:]   ## nodes
    y=FVCOM.variables['y'][:]
    xc=FVCOM.variables['xc'][:] ## elements
    yc=FVCOM.variables['yc'][:]
    nv = FVCOM.variables['nv'][:].T - 1  ## tri elemnents; offset for Python indexing.
    node = np.arange(1, x.size+1)
    nele = np.arange(1, xc.size+1)

### from FVCOM grid file (or serialized pickle file)
- `PyFVCOM.grid.Domain('hoge_grd.dat', 'cartesian', zone)` where zone is UTM zone number, e.g. '54'.
- `mesh = PyFVCOM.grid.Domain('tokyobay_futtsu5.0_grd.dat', 'cartesian', '54')`
- Grid data `mesh.grid.` lon, lat, lonc, latc, x, y, xc, yc, nv, open_boundaries, open_boundary_nodes
- Grid dimensions `mesh.dims.` node, nele, element, open_boundary, open_boundary_nodes, elements

In [ ]:
if grid_is_fvcom:
    if os.path.isfile(fvcom_grid_pickle):
        ## If serialized file exists, it is to be opened.
        with open(fvcom_grid_pickle, "rb") as file:
            mesh = pickle.load(file)
            print("Reading the FVCOM grid from pickle. Ensure the serialized grid data is up-to-date.")
    else:
        ## First time, read fvcom_grid and serialized it into fvcom_grid_pickle.
        ## A bit slow; be patient.
        #pickle_file = fvcom_grid[:-3] + "pickle"
        print("Reading the FVCOM grid from", fvcom_grid, "and dumping into", fvcom_grid_pickle, "for efficiency." )
        mesh = pf.grid.Domain(fvcom_grid, 'cartesian', '54')
        with open(fvcom_grid_pickle, "wb") as file:
            pickle.dump(mesh, file)

In [ ]:
if grid_is_fvcom:
    x = mesh.grid.x
    y = mesh.grid.y
    xc = mesh.grid.xc
    yc = mesh.grid.yc
    nv = mesh.grid.triangles
    node = mesh.dims.node
    nele = mesh.dims.nele

## Converting between UTM and geographic coordinates using [Proj](https://pyproj4.github.io/pyproj/stable/api/proj.html)
- `p=Proj(proj='utm', zone=54, ellps='WGS84')`
- `lons, lats = p(xs, ys, inverse=True)`
- `xs, ys = p(lons, lats)`

In [ ]:
zone = 54
hemi='N'
if hemi == 'S':
    y  = y  - 10000000
    yc = yc - 10000000
## Define coordinate converter p
p = Proj(proj='utm', zone=zone, ellps='WGS84')
lon, lat = p(x, y, inverse=True)
lonc, latc = p(xc, yc, inverse=True) 

## Create besttrack data using [besttracks](https://github.com/jsasaki-utokyo/besttracks)
Set typhoon ID in YYYYNN where NN is the typhoon number in the year of YYYY.

In [ ]:
fpath = "/home/teem/Github/besttracks/data/jma_rsmc/bst_all.txt"  ## Specify JMA best track data path
ID = '201919'  ## Set typhoon ID
df = parseJMA(fpath)
# print(df.head())
df = df[df.ID == ID]
df = df.set_index('TIME')
# df = df.tz_localize('UTC')  ## Set time zone as UTC
df = df.resample('H').interpolate()  ## Up-sampling in every hour with interpolation 
XUTM, YUTM = p(df.LON.values, df.LAT.values, inverse=False)  ## convert to UTM using Proj()
df['XUTM'] = XUTM
df['YUTM'] = YUTM
#print(df.PRS.values[240])

### Calculate translation speed
The last row is a copy of the previous row so that the number of rows does not change. 

In [ ]:
TRANSLATION = np.sqrt(np.square(XUTM[1:]-XUTM[:-1]) + np.square(YUTM[1:]-YUTM[:-1])) / 3600
TRANSLATION = np.append(TRANSLATION, TRANSLATION[-1])  ## Append the last row
df['TRANSLATION'] = TRANSLATION
df.fillna(method='ffill', inplace=True)
# print(df.TRANSLATION.values[10])
df.head()

### Extract best track parameters for parametric typhoon model

In [ ]:
xt_c = df.XUTM.values  ## typhoon track center x in UTM
yt_c = df.YUTM.values  ## typhoon track center y in UTM
sp_c  = df.PRS.values   ## typhoon track central surface pressure in hPa
translation_speed = df.TRANSLATION.values
num_tracks = xt_c.size - 1 ## should be -1 to obtain typhoon translation vector (one more track required)
timestamps = df.index

## Prepare time variable consistent with that in FVCOM
FVCOM uses modified julian day of "days since 1858-11-17 00:00:00" but calendar should be gregorian.

In [ ]:
def days_since_calendar_date(timestamp, calendar_date='1858-11-17 00:00:00'):
    '''
    Calculate days in real since calendar_date in UTM.
    
    Parameters
    ----------
    timestamp: pandas Timestamp
    calendar_date: str
        Default modified julian day in FVCOM
    
    Returns
    -------
    days: real
    '''
    calendar_date = parse(calendar_date)
    #print('calendar_date = ', calendar_date)
    days_int = (timestamp - calendar_date).days
    within_day = ((timestamp - calendar_date).seconds / (24*3600))
    days = days_int + within_day
    return days

## Storm model netCDF definition
- Use **NETCDF4** (default). The following is only for reference.
- In the case of very huge data, NETCDF3_64BIT_OFFSET would not work by raising a strange error. NETCDF4_CLASSIC works.
- FVCOM with netCDF-3.6.3 may need NETCDF3_64BIT_OFFSET.
- Options: NETCDF4 (default), NETCDF4_CLASSIC (use the version 4 disk format (HDF) but omits features not found in the verion 3 API), NETCDF3_64BIT_OFFSET (introduced in version 3.6.0, allowing file sizes greater than 2GB)

In [ ]:
def netcdf_storm(ncfile, x, y, lon, lat, xc, yc, lonc, latc, nv, format="NETCDF4",
                 title = "FVCOM storm forcing file",
                 institution = "The University of Tokyo",
                 source = "FVCOM grid (unstructured) surface forcing",
                 references = "http://fvcom.smast.umassd.edu, http://codfish.smast.umassd.edu",
                 Conventions = "CF-1.0",
                 units = "days since 1858-11-17 00:00:00", ## Modified Julian Day
                 CoordinateSystem = "Cartesian",
                 CoordinateProjection = "init=epsg:32654"):
    '''
    Create and define netCDF for storm model contining wind velocity and surface pressure.
    
    Parameters
    ----------
    ncfile(str): netCDF file name
    x(ndarray): 1D real FVCOM node
    y(ndarray): 1D real FVCOM node
    lon(ndarray): 1D real FVCOM node
    lat(ndarray): 1D real FVCOM node
    xc(ndarray): 1D real FVCOM nele
    yc(ndarray): 1D real FVCOM nele
    lonc(ndarray): 1D real FVCOM nele (cell center)
    latc(ndarray): 1D real FVCOM nele (cell center)
    nv(ndarray): 2D int FVCOM nv(three, nele)
    global_atts(dict): dictionary for global attributes
    format(str): netCDF format: NETCDF3_64BIT_OFFSET(default), NETCDF4_CLASSIC, NETCDF4
    
    Returns
    -------
    netCDF4.Dataset
    '''

    nc = netCDF4.Dataset(ncfile, "w", format=format)  ## Sometimes does not work with large data.
    # nc.set_fill_off()
    time = nc.createDimension("time", None)
    node = nc.createDimension("node", len(lat))
    nele  = nc.createDimension("nele", len(latc))
    three = nc.createDimension("three", 3)

    ## Coordinate variables
    times = nc.createVariable("time", "f8", ("time",))  ## f8 required; f4 causing inaccurate time
    nodes = nc.createVariable("node", "i4", ("node",))
    neles = nc.createVariable("nele", "i4", ("nele",))
    ## Variables
    xs = nc.createVariable("x", "f4", ("node",))
    ys = nc.createVariable("y", "f4", ("node",))
    lons = nc.createVariable("lon", "f4", ("node",))
    lats = nc.createVariable("lat", "f4", ("node",))
    xsc = nc.createVariable("xc", "f4", ("nele",))
    ysc = nc.createVariable("yc", "f4", ("nele",))
    lonsc = nc.createVariable("lonc", "f4", ("nele",))
    latsc = nc.createVariable("latc", "f4", ("nele",))
    nvs = nc.createVariable("nv", "i4", ("nele", "three",))

    sp = nc.createVariable("air_pressure", "f4", ("time", "node",))
    u_w_x  = nc.createVariable("u_w_x",  "f4", ("time", "nele",))
    u_w_y  = nc.createVariable("u_w_y",  "f4", ("time", "nele",))
    ## Global attributes
    nc.title = title
    nc.institution = institution
    nc.source = source
    nc.history = "created " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    nc.references = references
    nc.Conventions = Conventions
    nc.CoordinateSystem = CoordinateSystem
    nc.CoordinateProjection = CoordinateProjection
    ## Coordinate variable attributes
    times.units = "days since 1858-11-17 00:00:00" ## Modified Julian Day
    times.calendar = "standard"
    ## Variable attributes
    xs.long_name = "nodal x-coordinate"
    xs.units = "meters"
    ys.long_name = "nodal y-coordinate"
    ys.units = "meters"
    lons.long_name = "nodal longitude"
    lons.standard_name = "longitude"
    lons.units = "degrees_east"
    lats.long_name = "nodal latitude"
    lats.standard_name = "latitude"
    lats.units = "degrees_north"
    xsc.long_name = "zonal x-coordinate"
    xsc.units = "meters"
    ysc.long_name = "zonal y-coordinae"
    ysc.units = "meters"
    lonsc.long_name = "zonal longitude"
    lonsc.standard_name = "longitude"
    lonsc.units = "degrees_east"
    latsc.long_name = "zonal latitude"
    latsc.standard_name = "latitude"
    latsc.units = "degrees_north"
    nvs.long_name = "nodes surrounding element"
    sp.units = "hPa"
    sp.long_name = "surface air pressure"
    u_w_x.units = "m/s"
    u_w_x.long_name = "x component of wind velocity"
    u_w_y.units = "m/s"
    u_w_y.long_name = "y component of wind velocity"
    ## Writing coordinate variable data
    #print(lat)
    xs[:] = x
    ys[:] = y
    lats[:] = lat
    lons[:] = lon
    xsc[:] = xc
    ysc[:] = yc
    latsc[:] = latc
    lonsc[:] = lonc
    nvs[:,:] = nv

    return nc

# Create parametric model netCDF
### Add netCDF definitions for cyclonic and translation velocities in parametric model
These are not defined in ERA and Hybrid.

In [ ]:
nv.shape

In [ ]:
nc = netcdf_storm('parametric_model.nc', x, y, lon, lat, xc, yc, lonc, latc, nv)
u_w1_x = nc.createVariable("u_w1_x", "f4", ("time", "nele",))
u_w1_y = nc.createVariable("u_w1_y", "f4", ("time", "nele",))
u_w2_x = nc.createVariable("u_w2_x", "f4", ("time", "nele",))
u_w2_y = nc.createVariable("u_w2_y", "f4", ("time", "nele",))
u_w1_x.units = "m/s"
u_w1_x.long_name = "x component of cyclonic wind velocity"
u_w1_y.units = "m/s"
u_w1_y.long_name = "y component of cyclonic wind velocity"
u_w2_x.units = "m/s"
u_w2_x.long_name = "x component of translation velocity"
u_w2_y.units = "m/s"
u_w2_y.long_name = "y component of translation velocity"

## Surface pressure of Myers model

In [ ]:
P_0 = 1013.25  # ambient or environmental pressure in hPa
r_min = 10
r_max = (1.633 * sp_c - 1471.35) * 1000  ## when sp_c >= 950 hPa
r_max_tmp = (0.769 * sp_c - 650.55) * 1000  ## when 880 hPa <= sp_c < 950 hPa
index_sp_c_lt_950, = np.where(sp_c < 950)  ## array index where sp_c < 950 hPa 
r_max[index_sp_c_lt_950] = r_max_tmp[index_sp_c_lt_950]
index_sp_c_lt_880, = np.where(sp_c < 880)
#index_sp_c_lt_880.size
if index_sp_c_lt_880.size > 0:
    formatted = f"WARNING: sp_c at indices of {index_sp_c_lt_880} < 880 hPa"
    print(formatted)

In [ ]:
## sp: atm pressure at each node at i_track
for i_track in np.arange(num_tracks):  ## loop for tracks
    r = np.sqrt(np.square(x - xt_c[i_track].item()) + np.square(y - yt_c[i_track].item()))
    sp = np.where(r > r_min, (sp_c[i_track]+(P_0-sp_c[i_track]) * np.exp(-r_max[i_track]/r)), sp_c[i_track])
    nc.variables["air_pressure"][i_track, :] = sp
    nc.variables["time"][i_track] = days_since_calendar_date(timestamps[i_track])

## Wind field of Mitsuta-Fujii's model (1987)
- Referred to [Tajima et al. (2016)](https://doi.org/10.1142/S0578563416400027)
- `x_c`, `y_c` (`latc`): at elements.
- `xt_c` and `yt_c`: at typhoon track center
- Conversion from hPa to Pa is required by multiplying 100.

$$\boldsymbol{U_{\text{w}}(r, \theta)} = \boldsymbol{U_{\text{w1}}(r, \theta)} + \boldsymbol{U_{\text{w2}}(r, \theta)}$$
$$U_{\text{w1}}(r) = C_1 \left( -\frac{f_c r}{2} + \sqrt{\left( \frac{f_c r}{2} \right)^2 + \frac{r}{\rho_a} \frac{\partial P_a}{\partial r}} \right)$$
$$P_a(r) = P_c + (P_0 - P_c) \exp\left(-\frac{r_{\text{max}}}{r} \right) $$
$$\frac{\partial P_a}{\partial r} = (P_0 - P_c) \frac{r_{\text{max}}}{r^2} \exp\left( -\frac{r_{\text{max}}}{r} \right)$$
$$\left. \frac{\partial P_a}{\partial r}\right |_{r=r_{\text{max}}} = (P_0 - P_c)e^{-1}/r_{\text{max}}$$
$$\boldsymbol{U_{\text{w2}}(r, \theta)} = C_2 \frac{U_{\text{w1}}(r)}{U_{\text{w1}}(r_{\text{max}})}\boldsymbol{V_T}$$ 

where $\boldsymbol{V_T}$ is the translation vector, $f_c$ is the Coriolis parameter. Suppose $u_{\text{w1x}}$ and $u_{\text{w1y}}$ are the tangential $x$ and $y$ components of $U_{\text{w1}}(r)$,  $u_{\text{w1x}} = -(y_{\text{c}}-y_{\text{tc}})/r \times U_{\text{w1}}(r)$ and $u_{\text{w1y}} = (x_{\text{c}}-x_{\text{tc}})/r \times U_{\text{w1}}(r)$. Further, the velocity deflected with $\alpha$ anticlockwise:
$$ u_{\text{w1x}} \cos \alpha - u_{\text{w1y}} \sin \alpha$$ for x direction and $$u_{\text{w1x}} \sin \alpha + u_{\text{w1y}} \cos \alpha$$


In [ ]:
C1=2/3
C2=2/3
rho_a=1.225
omega = 2 * np.pi/(24*3600) ## angular frequency of earth rotation
alpha = 30 ## anticlockwise angle in dgree from the tangential direction of wind

alpha = np.pi*alpha/180  ## degree -> radian
for i_track in np.arange(num_tracks):  ## loop for typhoon tracks
    r = np.sqrt(np.square(xc - xt_c[i_track]) + np.square(yc - yt_c[i_track]))
    r = np.where(r >= r_min, r, r_min)  ## check: originally np.where(r >= r_min, r-r_min, r_min) but r-r_min may be zero. 
    f_c = 2 * omega * np.sin(np.pi*latc/180)
    dpa_dr = (P_0 - sp_c[i_track]) * 100 * r_max[i_track]/np.square(r) * np.exp(-r_max[i_track]/r)
    u_w1 = C1 * (-f_c*r/2 + np.sqrt(np.square(f_c*r/2) + r/rho_a * dpa_dr))
    u_gr = C1 * (-f_c*r/2 + np.sqrt(np.square(f_c*r/2) + 100 * (P_0 - sp_c[i_track])*r_max[i_track]/(rho_a*r) * np.exp(-r_max[i_track]/r) ))
    dpa_dr_r_max = (P_0 - sp_c[i_track]) * 100 * np.exp(-1)/r_max[i_track]  ## multply 100 to convert from hPa to Pa
    ## tangential direction
    u_w1_x_tmp = -(yc - yt_c[i_track])/r * u_w1  # x-component
    u_w1_y_tmp =  (xc - xt_c[i_track])/r * u_w1  # y-component
    ## rotating 30 degrees anticlockwise from the tangential direction
    u_w1_x = u_w1_x_tmp * np.cos(alpha) - u_w1_y_tmp * np.sin(alpha)
    u_w1_y = u_w1_x_tmp * np.sin(alpha) + u_w1_y_tmp * np.cos(alpha)
    
    u_w1_r_max = C1*(-f_c*r_max[i_track]/2 + np.sqrt(np.square(f_c*r_max[i_track]/2) + r_max[i_track]/rho_a * dpa_dr_r_max))
    u_w2 = C2 * u_w1/u_w1_r_max * translation_speed[i_track]
    
    dr_t = np.sqrt(np.square(xt_c[i_track+1]-xt_c[i_track]) + np.square(yt_c[i_track+1]-yt_c[i_track]))
    u_w2_x = (xt_c[i_track+1] - xt_c[i_track])/dr_t * u_w2
    u_w2_y = (yt_c[i_track+1] - yt_c[i_track])/dr_t * u_w2
    
    u_w_x = u_w1_x + u_w2_x
    u_w_y = u_w1_y + u_w2_y

    nc.variables["u_w1_x"][i_track, :] = u_w1_x
    nc.variables["u_w1_y"][i_track, :] = u_w1_y
    nc.variables["u_w2_x"][i_track, :] = u_w2_x
    nc.variables["u_w2_y"][i_track, :] = u_w2_y
    nc.variables["u_w_x"][i_track, :] = u_w_x
    nc.variables["u_w_y"][i_track, :] = u_w_y

## Close parametric model netCDF

In [ ]:
nc.close()

## Plot for checking parametric model
### Open parametric model netCDF

In [ ]:
ncfile = "parametric_model.nc"
nc = netCDF4.Dataset(ncfile, "r")

In [ ]:
i_track = 120
sp = nc.variables["air_pressure"][i_track,:]
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
tp = ax.tripcolor(lon, lat, nv, sp, cmap='viridis', shading='flat')
#str_datetime = "day=" + str(time[num_tracks-1])
str_datetime = "day=" + str(num2date(nc['time'][num_tracks-1], units=nc['time'].units, calendar=nc['time'].calendar))
ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.2)
cb = fig.colorbar(tp, cax=cax)
cb.set_label("Atmospheric pressure (hPa)", fontsize=20)

In [ ]:
## Set plot parameters
i_track = 120
sp = nc.variables["air_pressure"][i_track,:]
u_w1_x = nc.variables["u_w1_x"][i_track,:]
u_w1_y = nc.variables["u_w1_y"][i_track,:]
u_w2_x = nc.variables["u_w2_x"][i_track,:]
u_w2_y = nc.variables["u_w2_y"][i_track,:]
u_w_x = nc.variables["u_w_x"][i_track,:]
u_w_y = nc.variables["u_w_y"][i_track,:]

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
u_w1_scale = np.sqrt(np.square(u_w1_x) + np.square(u_w1_y))
#vel_scale = u_w1_scale
#u_w2_scale = np.sqrt(np.square(u_w2_x) + np.square(u_w2_y))
#vel_scale = u_w2_scale
#u_w_scale = np.sqrt(np.square(u_w_x) + np.square(u_w_y))
#vel_scale = u_w_scale

vel_scale = 10
vel_scale = np.where(vel_scale > 0.1, vel_scale, 0.1)

ax.quiver(lonc, latc, u_w1_x/vel_scale, u_w1_y/vel_scale, cmap='jet', scale=100)
#ax.quiver(lonc, latc, u_w2_x/vel_scale, u_w2_y/vel_scale, cmap='jet', scale=100)
#ax.quiver(lonc, latc, u_w_x/vel_scale, u_w_y/vel_scale, cmap='jet', scale=100)

str_datetime = "day=" + str(num2date(nc['time'][i_track], units=nc['time'].units, calendar=nc['time'].calendar))
ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)  ## Put text relative to the axis frame

ax.set_xlim(137,142)
ax.set_ylim(18, 25)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

## Plot atmospheric pressure field
tp = ax.tripcolor(lon, lat, nv, sp, cmap='viridis', shading='flat')
str_datetime = "day=" + str(num2date(nc['time'][num_tracks-1], units=nc['time'].units, calendar=nc['time'].calendar))
#str_datetime = "day=" + str(nc['time'][num_tracks-1])
ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)  ## Put text relative to the axis frame
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.2)
cb = fig.colorbar(tp, cax=cax)
cb.set_label("Atmospheric pressure (hPa)", fontsize=20)

## Plot vectors
u_w1_scale = np.sqrt(np.square(u_w1_x) + np.square(u_w1_y))
vel_scale = 10
vel_scale = np.where(vel_scale > 0.1, vel_scale, 0.1)
ax.quiver(lonc, latc, u_w1_x/vel_scale, u_w1_y/vel_scale, scale=100)

ax.set_xlim(137,142)
ax.set_ylim(18, 25)

## Grid layout plot ([reference](https://www.yutaka-note.com/entry/matplotlib_subplots))
### Using `plt.subplots` but difficult in fine adjustment
- `squeeze=False`: The shape of axes is kept 2D even for the cases of one row.

In [ ]:
'''
nrows=1
ncols=3
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18,6), squeeze=False, 
                         sharey="all")
str_datetime = "day=" + str(time[num_tracks-1])
u_w1_scale = np.sqrt(np.square(u_w1_x) + np.square(u_w1_y))
vel_scale = 10
vel_scale = np.where(vel_scale > 0.1, vel_scale, 0.1)
for i in range(nrows):
    for j in range(ncols):
        ## Plot atmospheric pressure field
        tp = axes[i,j].tripcolor(lon, lat, nv, sp, cmap='viridis', shading='flat')
        axes[i,j].text(0.05, 0.95, str_datetime, transform=axes[i,j].transAxes)  ## Put text relative to the axis frame
        axes[i,j].axis('equal')
        axes[i,j].set_xlim(137,142)
        axes[i,j].set_ylim(18, 25)
        ## PLot vector field
        axes[i,j].quiver(lonc, latc, u_w1_x/vel_scale, u_w1_y/vel_scale, scale=100)
cb = fig.colorbar(tp, ax=axes.flat)
cb.set_label("Atmospheric pressure (hPa)", fontsize=20)
'''

### Using ImageGrid (recommended) [Ref 1](https://sabopy.com/py/matplotlib-19/), [Ref 2](https://sabopy.com/py/matplotlib-20/), [Official ref](https://matplotlib.org/stable/api/_as_gen/mpl_toolkits.axes_grid1.axes_grid.ImageGrid.html#mpl_toolkits.axes_grid1.axes_grid.ImageGrid)

In [ ]:
## Prepare for vector field plot
vel_scale = 0.8
vel_scale = np.where(vel_scale > 0.1, vel_scale, 0.1)
u_plot=[]; v_plot=[]
u_plot.append(u_w1_x); v_plot.append(u_w1_y) ## circular wind vector field deflected by 30 degrees anticlockwise
u_plot.append(u_w2_x); v_plot.append(u_w2_y) ## translation wind field
u_plot.append(u_w_x) ; v_plot.append(u_w_y)  ## final wind vector field
## Prepare for text of date
#str_datetime = "day=" + str(nc['time'][num_tracks-1])
str_datetime = "day=" + str(num2date(nc['time'][num_tracks-1], units=nc['time'].units, calendar=nc['time'].calendar))

## Start plotting
fig = plt.figure(figsize=(18,6))
grid = ImageGrid(fig, 111, nrows_ncols=(1,3), axes_pad=0.25, share_all=True, aspect=True,
                 cbar_location="right", cbar_mode="single", cbar_size="5%", cbar_pad=0.2)
## Add data to image grid
for i, ax in enumerate(grid):
    ## Plot atmospheric pressure field
    tp = ax.tripcolor(lon, lat, nv, sp, cmap='viridis', shading='flat')
    ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)  ## Put text relative to the axis frame
    ax.set_xlim(137,142)
    ax.set_ylim(18, 25)
    ## PLot vector field
    #ax.quiver(lonc, latc, u_w1_x/vel_scale, u_w1_y/vel_scale, scale=100)
    ax.quiver(lonc, latc, u_plot[i]/vel_scale, v_plot[i]/vel_scale, scale=100)
## Add colorbar
cb=ax.cax.colorbar(tp)
cb.set_label("Atmospheric pressure (hPa)")

## Close parametric model netCDF

In [ ]:
nc.close()

# Create ERA netCDF

In [ ]:
ncfile = "era_model.nc"
nc = netcdf_storm(ncfile, x, y, lon, lat, xc, yc, lonc, latc, nv)

## Read ERA data
- `sp(time, latitude, longitude)`: surface pressure (Pa)
- `time`: hours since 1900-01-01 00:00:00.0 (standard)

In [ ]:
## Read ERA wind
era_uv_nc = "era5_10u10v_201910041014.nc"
ERA_uv = netCDF4.Dataset(era_uv_nc, 'r')
#lon_era_uv = np.ma.filled(ERA_uv.variables['longitude'][:])
#lat_era_uv = np.ma.filled(ERA_uv.variables['latitude'][:])
#time_era_uv = np.ma.filled(ERA_uv.variables['time'][:])
#u_era = np.ma.filled(ERA_uv.variables['u10'][:])
#v_era = np.ma.filled(ERA_uv.variables['v10'][:])
lon_era_uv = ERA_uv.variables['longitude'][:]
lat_era_uv = ERA_uv.variables['latitude'][:]
time_era_uv = ERA_uv.variables['time'][:]
u_era = ERA_uv.variables['u10'][:]
v_era = ERA_uv.variables['v10'][:]


num_lon_era_uv = lon_era_uv.size
num_lat_era_uv = lat_era_uv.size
num_time_era_uv = time_era_uv.size

## Read ERA sea surface pressure
era_sp_nc = "era5_sp_201910041014.nc"
ERA_sp = netCDF4.Dataset(era_sp_nc, 'r')
#lon_era_sp = np.ma.filled(ERA_sp.variables['longitude'][:])
#lat_era_sp = np.ma.filled(ERA_sp.variables['latitude'][:])
#time_era_sp = np.ma.filled(ERA_sp.variables['time'][:])
#sp_era = np.ma.filled(ERA_sp.variables['sp'][:])
lon_era_sp = ERA_sp.variables['longitude'][:]
lat_era_sp = ERA_sp.variables['latitude'][:]
time_era_sp = ERA_sp.variables['time'][:]
sp_era = ERA_sp.variables['sp'][:]

num_lon_era_sp = lon_era_sp.size
num_lat_era_sp = lat_era_sp.size
num_time_era_sp = time_era_sp.size

## Check whether both times are identical, or raise an error.
if np.all(time_era_uv == time_era_sp):
    print("time_era_uv == time_era_sp; use time_era")
    time_era = time_era_uv
    time_era_units = ERA_sp.variables['time'].units
    time_era_calendar = ERA_sp.variables['time'].calendar
    print(time_era_units)
    print(time_era_calendar)
else:
    raise ValueError("ERROR: time_era /= time_era_uv")

### Find a ERA grid point `(i,j)` where the grid area between `(i,j)` and `(i+1,j+1)` contains an FVCOM node or cell center
lon_era ranges from small to large numbers while lat_era ranges from large to small numbers.
```
    i       i+1    *: FVCOM grid node or cell center index n
j   |--------|     i: Longitude grid index of ERA   
    |  * n   |     j: Latitude grid indes of ERA (positive downward)
j+1 |--------|
```
### Interpolate ERA surface pressure at FVCOM node

In [ ]:
#dlon_era_uv = lon_era_uv[1] - lon_era_uv[0]
#dlat_era_uv = lat_era_uv[0] - lat_era_uv[1]
dlon_era_sp = lon_era_sp[1] - lon_era_sp[0]
dlat_era_sp = lat_era_sp[0] - lat_era_sp[1]
i_sp = (np.trunc((lon - lon_era_sp[0]) / dlon_era_sp)).astype(int)
j_sp = (np.trunc((lat_era_sp[0] - lat) / dlat_era_sp)).astype(int)
#print(lat_era_sp[0])
#print(lat)
#print(j_sp)
#print(lat[0])
#print(lat_era_sp[j_sp[0]], lat_era_sp[j_sp[0]+1])
cx0 = (lon - lon_era_sp[i_sp]) / (lon_era_sp[i_sp+1] - lon_era_sp[i_sp])
cx1 = (lon_era_sp[i_sp + 1] - lon) / (lon_era_sp[i_sp + 1] - lon_era_sp[i_sp])
cy0 = (lat_era_sp[j_sp] - lat) / (lat_era_sp[j_sp] - lat_era_sp[j_sp + 1])
cy1 = (lat - lat_era_sp[j_sp + 1]) / (lat_era_sp[j_sp] - lat_era_sp[j_sp + 1])
#print(cy0 + cy1)

#print(sp_era[0, [0, 3], [0, 2]])
#print(sp_era[0, 0, 0])
#print(sp_era[0, 3, 2])

In [ ]:
dates_era = num2date(time_era, units=time_era_units, calendar=time_era_calendar)
days_era = date2num(dates_era, units="days since 1858-11-17 00:00:00", calendar="gregorian")

In [ ]:
## Interpolation
#print(cy0.shape)
#print(cy1.shape)
#print(sp_era_fvcom_j0.shape)
#print(sp_era_fvcom_j1.shape)
## ERAの時間とFVCOMの時間を合わせ，netCDFはFVCOMの時間で作る
## Matching ERA time and FVCOM time; creating netCDF time using FVCOM time
for time in np.arange(time_era.size):
    sp_era_fvcom_j0 = cx1 * sp_era[time, j_sp, i_sp] + cx0 * sp_era[time, j_sp, i_sp+1]
    #print(sp_era_fvcom_j0.shape)
    sp_era_fvcom_j1 = cx1 * sp_era[time, j_sp+1, i_sp] + cx0 * sp_era[time, j_sp+1, i_sp+1]
    #print(sp_era_fvcom_j1.shape)
    sp_era_fvcom = cy1 * sp_era_fvcom_j0 + cy0 * sp_era_fvcom_j1
    #print(sp_era_fvcom.shape)
    nc.variables["time"][time] = days_era[time]
    nc.variables["air_pressure"][time, :] = sp_era_fvcom

### Interpolate ERA wind field at FVCOM cell center

In [ ]:
dlon_era_uv = lon_era_uv[1] - lon_era_uv[0]
dlat_era_uv = lat_era_uv[0] - lat_era_uv[1]
i_uv = (np.trunc((lonc - lon_era_uv[0]) / dlon_era_uv)).astype(int)
j_uv = (np.trunc((lat_era_uv[0] - latc) / dlat_era_uv)).astype(int)
cx0 = (lonc - lon_era_uv[i_uv]) / (lon_era_uv[i_uv+1] - lon_era_uv[i_uv])
cx1 = (lon_era_uv[i_uv + 1] - lonc) / (lon_era_uv[i_uv + 1] - lon_era_uv[i_uv])
cy0 = (lat_era_uv[j_uv] - latc) / (lat_era_uv[j_uv] - lat_era_uv[j_uv + 1])
cy1 = (latc - lat_era_uv[j_uv + 1]) / (lat_era_uv[j_uv] - lat_era_uv[j_uv + 1])
                                                                      
for time in np.arange(time_era.size):
    u_era_fvcom_j0 = cx1 * u_era[time, j_uv, i_uv] + cx0 * u_era[time, j_uv, i_uv+1]
    #print(sp_era_fvcom_j0.shape)
    u_era_fvcom_j1 = cx1 * u_era[time, j_uv+1, i_uv] + cx0 * u_era[time, j_uv+1, i_uv+1]
    #print(sp_era_fvcom_j1.shape)
    u_era_fvcom = cy1 * u_era_fvcom_j0 + cy0 * u_era_fvcom_j1
    #print(sp_era_fvcom.shape)
    nc.variables["u_w_x"][time, :] = u_era_fvcom

    v_era_fvcom_j0 = cx1 * v_era[time, j_uv, i_uv] + cx0 * v_era[time, j_uv, i_uv+1]
    v_era_fvcom_j1 = cx1 * v_era[time, j_uv+1, i_uv] + cx0 * v_era[time, j_uv+1, i_uv+1]
    v_era_fvcom = cy1 * v_era_fvcom_j0 + cy0 * v_era_fvcom_j1
    nc.variables["u_w_y"][time, :] = v_era_fvcom

## Close ERA netCDF

In [ ]:
nc.close()

## Plot for checking ERA
### Open ERA netCDF

In [ ]:
ncfile = "era_model.nc"
nc = netCDF4.Dataset(ncfile, "r")

### Plot ERA surface pressure field

In [ ]:
timestep = 180
sp = nc.variables["air_pressure"][timestep,:]
u_w_x = nc.variables["u_w_x"][timestep,:]
u_w_y = nc.variables["u_w_y"][timestep,:]
str_datetime = "day=" + str(dates_era[timestep])

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

## Plot atmospheric pressure field
tp = ax.tripcolor(lon, lat, nv, sp/100, cmap='viridis', shading='flat')
ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)  ## Put text relative to the axis frame
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.2)
cb = fig.colorbar(tp, cax=cax)
cb.set_label("Atmospheric pressure (hPa)", fontsize=20)

#ax.set_xlim(137,142)
#ax.set_ylim(18, 25)

### Plot ERA wind and surface pressure field

In [ ]:
timestep = 180
sp = nc.variables["air_pressure"][timestep,:]
u_w_x = nc.variables["u_w_x"][timestep,:]
u_w_y = nc.variables["u_w_y"][timestep,:]
str_datetime = "day=" + str(dates_era[timestep])

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

## Plot surface pressure
tp = ax.tripcolor(lon, lat, nv, sp/100, cmap='viridis', shading='flat')
ax.text(0.05, 0.95, str_datetime, transform=ax.transAxes)  ## Put text relative to the axis frame
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.2)
cb = fig.colorbar(tp, cax=cax)
cb.set_label("Atmospheric pressure (hPa)", fontsize=20)

## Plot vectors
## Uniform arrow length
vel_scale = 0.5 * np.sqrt(np.square(u_w_x) + np.square(u_w_y))
vel_scale = np.where(vel_scale > 0.1, vel_scale, 0.1)  ## Avoid divided by zero
## Proportional arrow lenght to speed
#vel_scale = 12
ax.quiver(lonc, latc, u_w_x/vel_scale, u_w_y/vel_scale, scale=100)

ax.set_xlim(132,142)
ax.set_ylim(25, 35)

## Close ERA netCDF

In [ ]:
nc.close()

# Create hybrid netCDF

## Open parametric model netCDF and ERA netCDF
- Extract the start (`ids`) and end (`ide`) indices of the common time range for PRM (parametric) and ERA
- Extract the same time range (from `datetime_s` to `datetime_e`) for besttrack DataFrame

In [ ]:
## Create hybrid netCDF
nc = netcdf_storm('hybrid_model.nc', x, y, lon, lat, xc, yc, lonc, latc, nv)

PRM = netCDF4.Dataset("parametric_model.nc", "r")
ERA = netCDF4.Dataset("era_model.nc", "r")
time_PRM = PRM.variables["time"]
time_ERA = ERA.variables["time"]
# print(num2date(time_PRM[:], units=time_PRM.units, calendar=time_PRM.calendar))
# print(num2date(time_ERA[:], units=time_ERA.units, calendar=time_ERA.calendar))
day_s, day_e = max(time_PRM[0], time_ERA[0]).item(), min(time_PRM[-1], time_ERA[-1]).item()
print(day_s, day_e)
ids_PRM, ide_PRM = np.where(time_PRM[:] == day_s)[0].item(), np.where(time_PRM[:] == day_e)[0].item()
print(ids_PRM, ide_PRM)
ids_ERA, ide_ERA = np.where(time_ERA[:] == day_s)[0].item(), np.where(time_ERA[:] == day_e)[0].item()
print(ids_ERA, ide_ERA)
datetime_s = str(num2date(time_PRM[ids_PRM], units=time_PRM.units, calendar=time_PRM.calendar))
datetime_e = str(num2date(time_PRM[ide_PRM], units=time_PRM.units, calendar=time_PRM.calendar))
# print(datetime_s, datetime_e)
# df.loc[datetime_s : datetime_e]

### Set inner (`r_i`) and outer (`r_o`) radii and band width (`b_w = r_o - r_i`) in meters

In [ ]:
r_i = 300000; r_o = 500000; b_w = r_o - r_i

In [ ]:
n = -1
for n_PRM in np.arange(ide_PRM-ids_PRM+1):
    n += 1
    ## Gets ERA time index (n_ERA) corresponding to 0-indexed PRM's n
    n_ERA = n_PRM + ids_ERA - ids_PRM
    datetime_BST = str(num2date(time_PRM[n_PRM], units=time_PRM.units, calendar=time_PRM.calendar))
    xt_c = df.loc[datetime_BST, 'XUTM']  ## x (UTM) of typhoon center
    yt_c = df.loc[datetime_BST, 'YUTM']  ## y (UTM) of typhoon center
    dist_sp = np.sqrt(np.square(x  - xt_c) + np.square(y  - yt_c))
    dist_uv = np.sqrt(np.square(xc - xt_c) + np.square(yc - yt_c))

    sp = ERA['air_pressure'][n_ERA, :]
    sp = np.where(dist_sp <= r_i, PRM['air_pressure'][n_PRM, :], sp)
    ## dist_sp = dist_sp when FVCOM node is in the region between inner and outer circles, otherwise = np.nan
    dist_sp = np.where((dist_sp > r_i) & (dist_sp < r_o), dist_sp, np.nan)  ## dist_sp between inner and outer circles
    dr_i = dist_sp - r_i  ## distance between FVCOM True node and inner circle
    dr_o = r_o - dist_sp  ## distance between FVCOM True node and outer circle
    sp = np.where(~np.isnan(dist_sp), dr_i/b_w * ERA['air_pressure'][n_ERA,:]
                  + dr_o/b_w * PRM['air_pressure'][n_PRM,:], sp)

    nc.variables["time"][n] = PRM["time"][n_PRM]
    nc.variables["air_pressure"][n, :] = sp

    u_w_x = ERA['u_w_x'][n_ERA, :]
    u_w_y = ERA['u_w_y'][n_ERA, :]
    u_w_x = np.where(dist_uv <= r_i, PRM['u_w_x'][n_PRM, :], u_w_x)
    u_w_y = np.where(dist_uv <= r_i, PRM['u_w_y'][n_PRM, :], u_w_y)
    ## dist_uv = dist_uv when FVCOM nele is in the region between inner and outer circles, otherwise = np.nan
    dist_uv = np.where((dist_uv > r_i) & (dist_uv < r_o), dist_uv, np.nan)  ## dist_uv between inner and outer circles
    dr_i = dist_uv - r_i  ## distance between FVCOM True nele and inner circle
    dr_o = r_o - dist_uv  ## distance between FVCOM True nele and outer circle
    u_w_x = np.where(~np.isnan(dist_uv), dr_i/b_w * ERA['u_w_x'][n_ERA,:] + dr_o/b_w * PRM['u_w_x'][n_PRM,:], u_w_x)
    u_w_y = np.where(~np.isnan(dist_uv), dr_i/b_w * ERA['u_w_y'][n_ERA,:] + dr_o/b_w * PRM['u_w_y'][n_PRM,:], u_w_y)
    nc.variables["u_w_x"][n, :] = u_w_x
    nc.variables["u_w_y"][n, :] = u_w_y

## Close hybrid netCDF

In [ ]:
PRM.close()
ERA.close()
nc.close()

# Plot for checking hybrid

# Create FVCOM input netCDF
## Open storm model netCDF

In [ ]:
ncifile = 'parametric_model.nc'
#ncifile = 'era_model.nc'
#ncifile = 'hybrid_model.nc'
nci = netCDF4.Dataset(ncifile, "r")

In [ ]:
nele = nci.dimensions['nele'].size
node = nci.dimensions['node'].size
x = nci['x'][:]
y = nci['y'][:]
lon = nci['lon'][:]
lat = nci['lat'][:]
xc = nci['xc'][:]
yc = nci['yc'][:]
lonc = nci['lonc'][:]
latc = nci['latc'][:]
uwind_speed = nci['u_w_x'][:]
vwind_speed = nci['u_w_y'][:]
air_pressure = nci['air_pressure'][:]

In [ ]:
global_attributes = nci.__dict__  ## global attribute {name: value} pairs in a dictionary
times = num2date(nci['time'][:], units=nci['time'].units, calendar=nci['time'].calendar)
#times = np.ma.filled(times, np.nan)
#times = [str(time) for time in times]
#times = [time.strftime() for time in times]
Times = [parse(time.strftime()) for time in times]
#print(Times)
#Times[0].strftime('%Y-%m-%dT%H:%M:%S.%f')
#print(times[0].strftime('%Y-%m-%d %H:%M:%S.%f'))
#print(times[0].strftime('%Y-%m-%dT%H:%M:%S'))
#print(times[0].strftime())
#print(uwind_speed.shape)
#print(times.shape)
#print(latc.shape)
#date2num(Times, units='days since 1858-11-17 00:00:00')
nci.close()

## Open and define FVCOM input netCDF
- `add_varialbe(self, name, data, dimensions, attributes, format, ncopts={})`

In [ ]:
def write_fvcom_forcing(ncfile, nele, node, global_attributes, nv, Times, x, y, lon, lat, xc, yc, lonc, latc,
                        uwind_speed, vwind_speed, air_pressure):
    '''
    
    '''
    dimensions = {'nele': nele, 'node': node, 'three': 3, 'time': None, 'DateStrLen': 26}
    with pf.preproc.WriteForcing(filename=ncfile, dimensions=dimensions, global_attributes=global_attributes,
                                 format='NETCDF4') as nc:
        nc.write_fvcom_time(time=Times)
        nc.add_variable('x', x, ('node',), {'long_name': 'nodal x-coordinate', 'units': 'meters'})
        nc.add_variable('y', y, ('node',), {'long_name': 'nodal y-coordinate', 'units': 'meters'})
        nc.add_variable('lon', lon, ('node',), {'long_name': 'nodal longitude', 'standard_name': 'longitude', \
                                                'units': 'degrees_east'})
        nc.add_variable('lat', lat, ('node',), {'long_name': 'nodal latitude', 'standard_name': 'latitude', \
                                                'units': 'degrees_north'})
        nc.add_variable('xc', xc, ('nele',), {'long_name': 'zonal x-coordinate', 'units': 'meters'})
        nc.add_variable('yc', yc, ('nele',), {'long_name': 'zonal y-coordinate', 'units': 'meters'})
        nc.add_variable('lonc', lonc, ('nele',), {'long_name': 'zonal longitude', 'standard_name': 'longitude', \
                                                  'units': 'degrees_east'})
        nc.add_variable('latc', latc, ('nele',), {'long_name': 'zonal latitude', 'standard_name': 'latitude', \
                                                  'units': 'degrees_north'})
        nc.add_variable('nv', nv.T, ('three', 'nele',), {'long_name': 'nodes surrounding element'}, 'i')

        nc.add_variable('uwind_speed', uwind_speed, ('time', 'nele',), {'long_name': 'Eastward Wind Speed', \
                        'standard_name': 'Wind Speed', 'units': 'm/s', 'grid': 'fvcom_grid', 'type': 'data'})
        nc.add_variable('vwind_speed', vwind_speed, ('time', 'nele',), {'long_name': 'Northward Wind Speed', \
                        'standard_name': 'Wind Speed', 'units': 'm/s', 'grid': 'fvcom_grid', 'type': 'data'})
        nc.add_variable('air_pressure', air_pressure, ('time', 'node',), {'long_name': 'surface air pressure', \
                        'units': 'Pa', 'grid': 'fvcom_grid', 'coordinates': 'FVCOM Cartesian coordinates', 'type': 'data'})

write_fvcom_forcing(fvcom_wnd, nele, node, global_attributes, nv, Times, x, y, lon, lat, xc, yc, lonc, latc, 
                    uwind_speed, vwind_speed, air_pressure)

In [ ]:
nc = netCDF4.Dataset(fvcom_wnd, "r")
nc

In [ ]:
nc.close()